In [1]:
import pandas as pd
import os
from datetime import timezone
import datetime
import time
import math
import copy
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine

In [3]:
MasterData = pd.read_csv('Master_Data_File.csv', index_col='index')
MasterData

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
index,,,,,,,,
0,1444348800,2015-10-09 00:00:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
1,1444348860,2015-10-09 00:01:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
2,1444348920,2015-10-09 00:02:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
3,1444348980,2015-10-09 00:03:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
4,1444349040,2015-10-09 00:04:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
...,...,...,...,...,...,...,...,...
3168102,1634428860,2021-10-17 00:01:00,BTCUSD,60877.53,60877.53,60851.85,60875.98,0.182190
3168103,1634428920,2021-10-17 00:02:00,BTCUSD,60875.98,60937.30,60854.84,60922.25,1.135391
3168104,1634428980,2021-10-17 00:03:00,BTCUSD,60922.25,60925.99,60809.75,60809.75,0.108527


In [138]:
last_date=0
dupliate_indices=[]
for i in range(len(MasterData)) :
    unix = MasterData.loc[i,'Unix Timestamp']
    if i !=0:
        difference = unix-last_date
        if (difference!=60):
            dupliate_indices.append(i)
            print('Difference: ', difference, ' index ',i)
    last_date=unix

Difference:  0  index  1510679
Difference:  0  index  1516283
Difference:  0  index  1632820
Difference:  0  index  1746185
Difference:  0  index  1759685
Difference:  0  index  1764008
Difference:  0  index  1781287
Difference:  0  index  1785610
Difference:  0  index  1814411
Difference:  0  index  1824492
Difference:  0  index  1841774
Difference:  0  index  1844654
Difference:  0  index  1864816
Difference:  0  index  1884977
Difference:  0  index  1915218
Difference:  0  index  1933939
Difference:  0  index  1959859
Difference:  0  index  1969940
Difference:  0  index  1994421
Difference:  0  index  2014583
Difference:  0  index  2034744
Difference:  0  index  2054905
Difference:  0  index  2075066
Difference:  0  index  2095227
Difference:  0  index  2125469
Difference:  0  index  2152830
Difference:  0  index  2165791
Difference:  0  index  2196031
Difference:  0  index  2206112
Difference:  0  index  2236352
Difference:  0  index  2256513
Difference:  0  index  2276674
Differen

In [139]:
MasterData = MasterData.drop(labels=dupliate_indices, axis=0)
MasterData = MasterData.reset_index(drop=True)

last_date=0

for i in range(len(MasterData)) :
    unix = MasterData.loc[i,'Unix Timestamp']
    if i !=0:
        difference = unix-last_date
        if (difference!=60):
            print('Difference: ', difference, ' index ',i)
    last_date=unix

In [142]:
MasterData.to_csv('Master_Data_File.csv', index=False)

In [141]:
def combine_time_frames(data, first_date, increment):
    unix_val  =0
    date_val  =''
    open_val  =0
    high_val  =-1
    low_val   =999999999
    close_val =0
    volume_val=0
    
    for x in range(increment):
        item_date = (first_date + datetime.timedelta(minutes=x))
        item_unix = item_date.replace(tzinfo=timezone.utc).timestamp()
        item_data = data.loc[data['Unix Timestamp'] == item_unix]
        volume_val+= item_data.Volume.values[0]
        if item_data.High.values[0] > high_val:
            high_val= item_data.High.values[0]
        if item_data.Low.values[0]  < low_val:
            low_val = item_data.Low.values[0]
        if x == 0: 
            open_val  = item_data.Open.values[0]
            
        if x == increment-1: 
            close_val = item_data.Close.values[0]
            date_val  = item_data.Date.values[0]
            unix_val  = item_data.values[0][0]
    
    return[unix_val,date_val,'BTCUSD',open_val,high_val,low_val,close_val,volume_val]

def make_new_time_frame(data_set, increment, time_frame_str):
    new_data_set=[]
    iterator = 0 
    for x in range(int(len(data_set)/increment)):
        first_date = datetime.datetime.utcfromtimestamp(data_set.iloc[iterator][0])
        new_row = combine_time_frames(data_set,first_date,increment)
        new_row.append(time_frame_str)
        new_data_set.append(new_row)
        iterator += increment
    return new_data_set

* 1    = 1  min 
* 5    = 5  min 
* 30   = 30 min
* 60   = 1  hour 
* 240  = 4  hour 
* 720  = 12 hour
* 1440 = 24 hour 


In [143]:
times={
    "5 Min"  : 5,
    "30 Min" : 30,
    "1 Hour" : 60,
    "4 Hour" : 240,
    "12 Hour": 720,
    "24 Hour": 1440}


for x in times: 
    combined_data = make_new_time_frame(MasterData,times[x],x)
    combined_df   = pd.DataFrame(combined_data, columns =['Unix Timestamp','Date','Symbol','Open','High','Low','Close','Volume', 'Time Frame'])
    combined_df.to_csv(x+'TimeFrameData'+'.csv', index=False)
    print(x,'written to csv')


5 Min written to csv
30 Min written to csv
1 Hour written to csv
4 Hour written to csv
12 Hour written to csv
24 Hour written to csv


In [150]:
five_min_data = pd.read_csv('24 HourTimeFrameData.csv')
five_min_data
# .iloc[633620-100:633620-40]

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Time Frame
0,1444435140,2015-10-09 23:59:00,BTCUSD,243.95,249.97,243.60,245.39,56.027841,24 Hour
1,1444521540,2015-10-10 23:59:00,BTCUSD,245.39,246.30,244.60,246.30,39.348777,24 Hour
2,1444607940,2015-10-11 23:59:00,BTCUSD,246.30,249.50,245.96,249.50,14.487783,24 Hour
3,1444694340,2015-10-12 23:59:00,BTCUSD,249.50,249.50,247.60,247.60,62.787480,24 Hour
4,1444780740,2015-10-13 23:59:00,BTCUSD,247.60,252.87,245.75,250.42,115.425756,24 Hour
...,...,...,...,...,...,...,...,...,...
2195,1634083140,2021-10-12 23:59:00,BTCUSD,57500.00,57688.88,53911.79,56019.08,1812.111067,24 Hour
2196,1634169540,2021-10-13 23:59:00,BTCUSD,56019.08,57780.80,54250.00,57382.21,1382.767620,24 Hour
2197,1634255940,2021-10-14 23:59:00,BTCUSD,57382.21,58500.02,56807.96,57319.00,1189.078324,24 Hour
2198,1634342340,2021-10-15 23:59:00,BTCUSD,57319.00,62898.00,56889.00,61665.41,3228.330492,24 Hour


In [172]:
MasterData.to_csv('Master_Data_File.csv', index=False)
MasterData

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1444348800,2015-10-09 00:00:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
1,1444348860,2015-10-09 00:01:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
2,1444348920,2015-10-09 00:02:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
3,1444348980,2015-10-09 00:03:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
4,1444349040,2015-10-09 00:04:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
...,...,...,...,...,...,...,...,...
3168001,1634428860,2021-10-17 00:01:00,BTCUSD,60877.53,60877.53,60851.85,60875.98,0.182190
3168002,1634428920,2021-10-17 00:02:00,BTCUSD,60875.98,60937.30,60854.84,60922.25,1.135391
3168003,1634428980,2021-10-17 00:03:00,BTCUSD,60922.25,60925.99,60809.75,60809.75,0.108527
3168004,1634429040,2021-10-17 00:04:00,BTCUSD,60809.75,60829.33,60716.28,60716.28,0.216773


In [159]:
# for getting ENv vars 
env_vars = {} # or dict {}
with open('env.txt') as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        key, value = line.strip().split('=', 1)
        env_vars[key]= value
print(env_vars)

{'HOST': '64.20.33.250', 'DB': 'st19945_tickerData', 'USER': 'st19945_tickerData', 'PASSWORD': 'CU2021'}


In [160]:
# create sqlalchemy engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user=env_vars['USER'],
                               pw=env_vars['PASSWORD'],
                               host=env_vars['HOST'],
                               db=env_vars['DB']))

In [173]:
################################################
# Do not run unless you are pushing the whole new data to DB 
################################################

MasterData.to_sql('BTC_Ticker_Data', con = engine, index=True, if_exists = 'append', chunksize = 1000)

pushing time frame data 

In [157]:
five_min_data = pd.read_csv('5 MinTimeFrameData.csv')
five_min_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Time Frame
0,1444349040,2015-10-09 00:04:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,5 Min
1,1444349340,2015-10-09 00:09:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,5 Min
2,1444349640,2015-10-09 00:14:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,5 Min
3,1444349940,2015-10-09 00:19:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,5 Min
4,1444350240,2015-10-09 00:24:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,5 Min
...,...,...,...,...,...,...,...,...,...
633596,1634427840,2021-10-16 23:44:00,BTCUSD,61022.39,61022.39,60956.08,60970.43,2.655541,5 Min
633597,1634428140,2021-10-16 23:49:00,BTCUSD,60970.43,60982.11,60913.83,60925.32,0.247314,5 Min
633598,1634428440,2021-10-16 23:54:00,BTCUSD,60925.32,60925.32,60872.72,60872.72,0.295242,5 Min
633599,1634428740,2021-10-16 23:59:00,BTCUSD,60872.72,60885.50,60864.75,60864.75,0.151575,5 Min


In [161]:
five_min_data.to_sql('BTC_Ticker_Data_5_Min', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [162]:
thirty_min_data = pd.read_csv('30 MinTimeFrameData.csv')
thirty_min_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Time Frame
0,1444350540,2015-10-09 00:29:00,BTCUSD,243.95,243.95,243.60,243.60,3.000000,30 Min
1,1444352340,2015-10-09 00:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,30 Min
2,1444354140,2015-10-09 01:29:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,30 Min
3,1444355940,2015-10-09 01:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,30 Min
4,1444357740,2015-10-09 02:29:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,30 Min
...,...,...,...,...,...,...,...,...,...
105595,1634421540,2021-10-16 21:59:00,BTCUSD,60942.14,60960.72,60686.69,60690.92,4.974507,30 Min
105596,1634423340,2021-10-16 22:29:00,BTCUSD,60690.92,60856.97,60530.80,60583.90,5.405133,30 Min
105597,1634425140,2021-10-16 22:59:00,BTCUSD,60583.90,60941.21,60583.90,60813.77,4.285375,30 Min
105598,1634426940,2021-10-16 23:29:00,BTCUSD,60813.77,60988.88,60723.70,60975.97,4.317270,30 Min


In [163]:
thirty_min_data.to_sql('BTC_Ticker_Data_30_Min', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [164]:
one_hour_data = pd.read_csv('1 HourTimeFrameData.csv')
one_hour_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Time Frame
0,1444352340,2015-10-09 00:59:00,BTCUSD,243.95,243.95,243.60,243.60,3.000000,1 Hour
1,1444355940,2015-10-09 01:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,1 Hour
2,1444359540,2015-10-09 02:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,1 Hour
3,1444363140,2015-10-09 03:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,1 Hour
4,1444366740,2015-10-09 04:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,1 Hour
...,...,...,...,...,...,...,...,...,...
52795,1634414340,2021-10-16 19:59:00,BTCUSD,61026.08,61078.42,60843.28,60856.39,40.042673,1 Hour
52796,1634417940,2021-10-16 20:59:00,BTCUSD,60856.39,61042.21,60410.51,60979.15,36.188282,1 Hour
52797,1634421540,2021-10-16 21:59:00,BTCUSD,60979.15,61294.68,60686.69,60690.92,34.234604,1 Hour
52798,1634425140,2021-10-16 22:59:00,BTCUSD,60690.92,60941.21,60530.80,60813.77,9.690508,1 Hour


In [165]:
one_hour_data.to_sql('BTC_Ticker_Data_1_Hour', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [166]:
four_hour_data = pd.read_csv('4 HourTimeFrameData.csv')
four_hour_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Time Frame
0,1444363140,2015-10-09 03:59:00,BTCUSD,243.95,243.95,243.60,243.60,3.000000,4 Hour
1,1444377540,2015-10-09 07:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,4 Hour
2,1444391940,2015-10-09 11:59:00,BTCUSD,243.60,243.75,243.60,243.63,1.000000,4 Hour
3,1444406340,2015-10-09 15:59:00,BTCUSD,243.63,244.00,243.63,244.00,8.145083,4 Hour
4,1444420740,2015-10-09 19:59:00,BTCUSD,244.00,244.06,243.70,244.06,3.662625,4 Hour
...,...,...,...,...,...,...,...,...,...
13195,1634371140,2021-10-16 07:59:00,BTCUSD,61624.84,62366.08,61555.95,61703.88,129.040234,4 Hour
13196,1634385540,2021-10-16 11:59:00,BTCUSD,61703.88,62028.03,61350.01,61622.48,158.043904,4 Hour
13197,1634399940,2021-10-16 15:59:00,BTCUSD,61622.48,61622.48,60468.00,60951.72,211.221619,4 Hour
13198,1634414340,2021-10-16 19:59:00,BTCUSD,60951.72,61205.99,60174.14,60856.39,188.134971,4 Hour


In [167]:
four_hour_data.to_sql('BTC_Ticker_Data_4_Hour', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [168]:
twelve_hour_data = pd.read_csv('12 HourTimeFrameData.csv')
twelve_hour_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Time Frame
0,1444391940,2015-10-09 11:59:00,BTCUSD,243.95,243.95,243.60,243.63,4.000000,12 Hour
1,1444435140,2015-10-09 23:59:00,BTCUSD,243.63,249.97,243.63,245.39,52.027841,12 Hour
2,1444478340,2015-10-10 11:59:00,BTCUSD,245.39,245.54,244.60,244.60,27.223703,12 Hour
3,1444521540,2015-10-10 23:59:00,BTCUSD,244.60,246.30,244.60,246.30,12.125075,12 Hour
4,1444564740,2015-10-11 11:59:00,BTCUSD,246.30,246.30,246.30,246.30,0.080999,12 Hour
...,...,...,...,...,...,...,...,...,...
4395,1634255940,2021-10-14 23:59:00,BTCUSD,57623.00,58090.65,56807.96,57319.00,654.101934,12 Hour
4396,1634299140,2021-10-15 11:59:00,BTCUSD,57319.00,60000.00,56889.00,59556.71,744.116808,12 Hour
4397,1634342340,2021-10-15 23:59:00,BTCUSD,59556.71,62898.00,59215.31,61665.41,2484.213684,12 Hour
4398,1634385540,2021-10-16 11:59:00,BTCUSD,61665.41,62366.08,61129.04,61622.48,438.366647,12 Hour


In [169]:
twelve_hour_data.to_sql('BTC_Ticker_Data_12_Hour', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [170]:
twenty_four_hour_data = pd.read_csv('24 HourTimeFrameData.csv')
twenty_four_hour_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Time Frame
0,1444435140,2015-10-09 23:59:00,BTCUSD,243.95,249.97,243.60,245.39,56.027841,24 Hour
1,1444521540,2015-10-10 23:59:00,BTCUSD,245.39,246.30,244.60,246.30,39.348777,24 Hour
2,1444607940,2015-10-11 23:59:00,BTCUSD,246.30,249.50,245.96,249.50,14.487783,24 Hour
3,1444694340,2015-10-12 23:59:00,BTCUSD,249.50,249.50,247.60,247.60,62.787480,24 Hour
4,1444780740,2015-10-13 23:59:00,BTCUSD,247.60,252.87,245.75,250.42,115.425756,24 Hour
...,...,...,...,...,...,...,...,...,...
2195,1634083140,2021-10-12 23:59:00,BTCUSD,57500.00,57688.88,53911.79,56019.08,1812.111067,24 Hour
2196,1634169540,2021-10-13 23:59:00,BTCUSD,56019.08,57780.80,54250.00,57382.21,1382.767620,24 Hour
2197,1634255940,2021-10-14 23:59:00,BTCUSD,57382.21,58500.02,56807.96,57319.00,1189.078324,24 Hour
2198,1634342340,2021-10-15 23:59:00,BTCUSD,57319.00,62898.00,56889.00,61665.41,3228.330492,24 Hour


In [171]:
twenty_four_hour_data.to_sql('BTC_Ticker_Data_24_Hour', con = engine,index=True, if_exists = 'append', chunksize = 1000)


Testing Below this line 
***************


In [131]:
testing_set = MasterData
testing_set = testing_set.drop(labels=dupliate_indices, axis=0)


In [137]:
print(len(MasterData))
print(len(testing_set))
testing_set = testing_set.reset_index(drop=True)

last_date=0
dupliate_indices=[]
for i in range(len(testing_set)) :
    unix = testing_set.loc[i,'Unix Timestamp']
#     date = datetime.datetime.utcfromtimestamp()
    if i !=0:
        difference = unix-last_date
        if (difference!=60):
            dupliate_indices.append(i)
            print('Difference: ', difference, ' index ',i)
    last_date=unix

testing_set

3168107
3168006


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1444348800,2015-10-09 00:00:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
1,1444348860,2015-10-09 00:01:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
2,1444348920,2015-10-09 00:02:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
3,1444348980,2015-10-09 00:03:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
4,1444349040,2015-10-09 00:04:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
...,...,...,...,...,...,...,...,...
3168001,1634428860,2021-10-17 00:01:00,BTCUSD,60877.53,60877.53,60851.85,60875.98,0.182190
3168002,1634428920,2021-10-17 00:02:00,BTCUSD,60875.98,60937.30,60854.84,60922.25,1.135391
3168003,1634428980,2021-10-17 00:03:00,BTCUSD,60922.25,60925.99,60809.75,60809.75,0.108527
3168004,1634429040,2021-10-17 00:04:00,BTCUSD,60809.75,60829.33,60716.28,60716.28,0.216773


In [ ]:
# first_date = datetime.datetime.utcfromtimestamp(temp_set.iloc[0][0])
# combine_time_frames(temp_set,first_date,30)


In [ ]:
# five_min_data = make_new_time_frame(temp_set,5,time_frame_str)
# for y in five_min_data:
#     print(y)

# five_min_df = pd.DataFrame(five_min_data, columns =['Unix Timestamp','Date','Symbol','Open','High','Low','Close','Volume', time_frame_str])
# five_min_df